In [ ]:
!pip install pandas openpyxl requests beautifulsoup4

# Extracción Informacion wol.jw.org - Semanas Reunión

In [ ]:
# ========================================
# SCRIPT: EXTRACTOR DE INFORMACIÓN WOL - VIDA Y MINISTERIO
# ========================================

# CONFIGURACIÓN DE FECHA (¡CAMBIA ESTO!)
ANIO_BUSQUEDA = 2025
MES_BUSQUEDA = 12   # 12 = Diciembre

# ========================================
# INSTALACIÓN DE DEPENDENCIAS (SOLO PARA COLAB)
# ========================================
try:
    import google.colab
    print("🔧 Instalando dependencias para Google Colab...")
    !pip install requests beautifulsoup4 pandas openpyxl -q
    print("✅ Dependencias instaladas correctamente")
except ImportError:
    print("ℹ️  Ejecutándose en entorno local (Jupyter/PC)")

# ========================================
# CÓDIGO PRINCIPAL
# ========================================

import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
import calendar

# --- NUEVA FUNCIÓN: GENERADOR DE URLs AUTOMÁTICO ---
def generar_urls_por_mes(anio, mes):
    print(f"📅 Generando URLs para: {mes}/{anio}")
    urls = []
    cal = calendar.Calendar()
    # Recorremos el calendario buscando los lunes (inicio de semana teocrática en WOL)
    for week in cal.monthdatescalendar(anio, mes):
        for day in week:
            # day.weekday() == 0 es Lunes.
            # Verificamos que el lunes pertenezca al mes o sea parte de la semana que cae en el mes
            if day.weekday() == 0 and (day.month == mes or (day + timedelta(days=6)).month == mes):
                # Usamos la URL de tipo FECHA (/dt/) que WOL redirige automáticamente al ID correcto
                url = f"https://wol.jw.org/es/wol/dt/r4/lp-s/{day.year}/{day.month:02d}/{day.day:02d}"
                if url not in urls:
                    urls.append(url)
    return urls

def extraer_informacion(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        # Requests sigue redirecciones automáticamente, así que las URLs de fecha funcionan
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        info = {
            "semana": "",
            "libro": "",
            "cancion_inicial": "",
            "tesoros": "",
            "segunda_cancion": "",
            "tercera_cancion": "",
            "titulos_maestros": [],
            "nvc_titulo1": "",
            "nvc_titulo2": "",
            "nvc_titulo3": "",
            "info_lectura_biblia": "",
            "info_estudio_libro": ""
        }

        # 1. EXTRAER SEMANA (Tu lógica original)
        semana_elem = soup.find('h1', id='p1')
        if semana_elem:
            info["semana"] = semana_elem.get_text(strip=True)
        else:
            # Fallback por si cambia el ID p1
            title = soup.find('title')
            if title: info["semana"] = title.get_text().split('|')[0]

        # 2. EXTRAER LIBRO DE ESTUDIO (Tu lógica original)
        libro_elem = soup.find('h2', id='p2')
        if libro_elem:
            strong_tags = libro_elem.find_all('strong')
            if strong_tags:
                info["libro"] = " ".join([s.get_text(strip=True) for s in strong_tags])
            else:
                info["libro"] = libro_elem.get_text(strip=True)

        # 3. EXTRAER CANCIONES (Tu lógica original)
        cancion_elem = soup.find('h3', id='p3')
        if cancion_elem:
            cancion_text = cancion_elem.get_text(strip=True)
            match = re.search(r'Canción\s+(\d+)', cancion_text)
            if match:
                info["cancion_inicial"] = f"Canción {match.group(1)}"

        todas_h3 = soup.find_all('h3')
        canciones_encontradas = []
        # Evitar duplicar la primera
        cancion_inicial_texto = cancion_elem.get_text(strip=True) if cancion_elem else ""

        for h3 in todas_h3:
            texto = h3.get_text(strip=True)
            if "Canción" in texto and texto != cancion_inicial_texto:
                match = re.search(r'Canción\s+(\d+)', texto)
                if match:
                    cancion_formateada = f"Canción {match.group(1)}"
                    if cancion_formateada not in canciones_encontradas:
                        canciones_encontradas.append(cancion_formateada)

        if len(canciones_encontradas) >= 1:
            info["segunda_cancion"] = canciones_encontradas[0]
        if len(canciones_encontradas) >= 2:
            info["tercera_cancion"] = canciones_encontradas[1]

        # 4. EXTRAER TESOROS DE LA BIBLIA (Tu lógica original)
        tesoros_elem = soup.find(['h5', 'h4', 'h3'], id='p5')
        if tesoros_elem:
            info["tesoros"] = tesoros_elem.get_text(strip=True)

        # ==============================================================================
        # 5. EXTRAER INFORMACIÓN DE LECTURA DE LA BIBLIA (CORREGIDO Y ROBUSTO)
        # ==============================================================================
        # En lugar de buscar id='tt28' o 'p17', buscamos el texto "(4 mins.)"
        # dentro de un párrafo. La lectura SIEMPRE dura 4 minutos.

        # Buscamos todos los párrafos que tengan "(4 mins.)"
        parrafos_lectura = soup.find_all(lambda tag: tag.name == "p" and "(4 mins.)" in tag.text)

        for p in parrafos_lectura:
            # Verificación extra: La lectura suele tener un enlace a la lección 'th' (teaching)
            # o está justo después de la sección Tesoros.
            texto_p = p.get_text(strip=True)

            # Limpiamos el texto para quitar html tags si quedan
            texto_limpio = texto_p.replace('\xa0', ' ')

            # Buscamos el patrón: "(4 mins.) Texto Bíblico (th lección X)"
            # A veces dice "(4 mins. o menos)"
            match = re.search(r'\(\d+\s+mins?.*?\)\s*(.+)', texto_limpio)

            if match:
                posible_lectura = match.group(1).strip()
                # Para estar seguros de que no es otra parte de 4 mins, verificamos si contiene "th lección"
                # o si parece una cita bíblica (contiene números y :)
                if "th lección" in posible_lectura or re.search(r'\d+:', posible_lectura):
                    info["info_lectura_biblia"] = posible_lectura
                    break # Encontramos la correcta, salimos del bucle

        # Si la lógica nueva falla, intentamos mantener compatibilidad con IDs viejos por si acaso
        if not info["info_lectura_biblia"]:
             lectura_div = soup.find('div', id='tt28')
             if lectura_div:
                p_elem = lectura_div.find('p') # El primer p dentro del div tt28
                if p_elem:
                    texto = p_elem.get_text(strip=True)
                    match = re.search(r'\(\d+\s+mins?.*?\)\s*(.+)', texto)
                    if match: info["info_lectura_biblia"] = match.group(1).strip()

        # ==============================================================================

        # 6. EXTRAER INFORMACIÓN DEL ESTUDIO DEL LIBRO (Tu lógica original)
        estudio_biblico_encontrado = False
        for h3 in soup.find_all('h3'):
            if h3.find('strong') and "Estudio bíblico de la congregación" in h3.get_text(strip=True):
                siguiente_div = h3.find_next_sibling('div')
                if siguiente_div:
                    p_elem = siguiente_div.find('p')
                    if p_elem:
                        texto_html = str(p_elem).replace('<em>', '').replace('</em>', '')
                        temp_soup = BeautifulSoup(texto_html, 'html.parser')
                        texto_completo = temp_soup.get_text(separator=' ', strip=True)
                        match = re.search(r'\(\d+\s+mins?\.\)\s*(.+)', texto_completo)
                        if match:
                            info["info_estudio_libro"] = match.group(1).strip()
                            estudio_biblico_encontrado = True
                            break

        if not estudio_biblico_encontrado:
            # Tu lógica de fallback
            for div in soup.find_all('div', class_=lambda x: x and 'du-margin-inlineStart' in str(x)):
                p_elem = div.find('p')
                if p_elem and '(30 mins.)' in p_elem.get_text():
                    texto_completo = p_elem.get_text(separator=' ', strip=True)
                    if any(libro in texto_completo.lower() for libro in ['lfb', 'lmd', 'lvs', 'bt']): # Agregué 'bt' por si acaso
                        texto_html = str(p_elem).replace('<em>', '').replace('</em>', '')
                        temp_soup = BeautifulSoup(texto_html, 'html.parser')
                        texto_completo = temp_soup.get_text(separator=' ', strip=True)
                        match = re.search(r'\(\d+\s+mins?\.\)\s*(.+)', texto_completo)
                        if match:
                            info["info_estudio_libro"] = match.group(1).strip()
                            break

        # 7. EXTRAER TÍTULOS DE SEAMOS MEJORES MAESTROS (Tu lógica original)
        maestros_element = None
        vida_cristiana_element = None

        for h2 in soup.find_all('h2'):
            texto = h2.get_text(strip=True)
            if "SEAMOS MEJORES MAESTROS" in texto:
                maestros_element = h2
            elif "NUESTRA VIDA CRISTIANA" in texto:
                vida_cristiana_element = h2

        if maestros_element and vida_cristiana_element:
            maestros_id = maestros_element.get('id', '')
            vida_id = vida_cristiana_element.get('id', '')

            if maestros_id and vida_id:
                # Extraer números de IDs de forma segura
                m_num_str = ''.join(filter(str.isdigit, maestros_id))
                v_num_str = ''.join(filter(str.isdigit, vida_id))

                maestros_num = int(m_num_str) if m_num_str else 0
                vida_num = int(v_num_str) if v_num_str else 1000

                for h3 in soup.find_all('h3'):
                    h3_id = h3.get('id', '')
                    h3_num_str = ''.join(filter(str.isdigit, h3_id))

                    if h3_num_str:
                        h3_num = int(h3_num_str)
                        if maestros_num < h3_num < vida_num and h3.find('strong'):
                            strong_text = h3.find('strong').get_text(strip=True)
                            if not strong_text.startswith("Canción"):
                                info["titulos_maestros"].append(strong_text)

        # 8. EXTRAER TÍTULOS DE NUESTRA VIDA CRISTIANA (Tu lógica original)
        titulos_nvc = []
        estudio_biblico_texto = None

        if vida_cristiana_element:
            vida_id = vida_cristiana_element.get('id', '')
            v_num_str = ''.join(filter(str.isdigit, vida_id))

            if v_num_str:
                vida_num = int(v_num_str)
                for h3 in soup.find_all('h3'):
                    h3_id = h3.get('id', '')
                    h3_num_str = ''.join(filter(str.isdigit, h3_id))

                    if h3_num_str:
                        h3_num = int(h3_num_str)
                        if h3_num > vida_num and h3.find('strong'):
                            strong_text = h3.find('strong').get_text(strip=True)
                            if "Estudio bíblico de la congregación" in strong_text:
                                estudio_biblico_texto = strong_text
                                break
                            elif not strong_text.startswith("Canción"):
                                titulos_nvc.append(strong_text)

        if not estudio_biblico_texto:
            for h3 in soup.find_all('h3'):
                if h3.find('strong') and "Estudio bíblico de la congregación" in h3.get_text(strip=True):
                    estudio_biblico_texto = h3.find('strong').get_text(strip=True)
                    break

        info["nvc_titulo1"] = titulos_nvc[0] if len(titulos_nvc) > 0 else ""
        info["nvc_titulo2"] = titulos_nvc[1] if len(titulos_nvc) > 1 else ""
        info["nvc_titulo3"] = estudio_biblico_texto if estudio_biblico_texto else "Estudio bíblico de la congregación"

        return info

    except Exception as e:
        print(f"Error al extraer información de {url}: {str(e)}")
        return None

def procesar_urls_wol(urls):
    datos = []
    # Usamos el año y mes para el nombre del archivo si es posible
    timestamp = datetime.now().strftime("%Y%m%d")
    nombre_archivo = f"Info_Reunion_VMC_{ANIO_BUSQUEDA}_{MES_BUSQUEDA}.xlsx"

    print("🔍 Extrayendo información de las páginas WOL...")
    print("-" * 50)

    for i, url in enumerate(urls, 1):
        try:
            print(f"📄 ({i}/{len(urls)}) Procesando: {url}")
            info = extraer_informacion(url)

            if info is None:
                print("   ❌ No se pudo extraer información")
                continue

            # ESTRUCTURA DE SALIDA ORIGINAL
            datos_basicos = {
                "Semana": info["semana"],
                "Libro": info["libro"],
                "Canción Inicial": info["cancion_inicial"],
                "Tesoros de la Biblia": info["tesoros"],
                "Segunda Canción": info["segunda_cancion"],
                "Tercera Canción": info["tercera_cancion"]
            }

            for j, titulo in enumerate(info["titulos_maestros"], 1):
                if j <= 4:
                    datos_basicos[f"Maestros Título {j}"] = titulo

            for j in range(len(info["titulos_maestros"]) + 1, 5):
                datos_basicos[f"Maestros Título {j}"] = ""

            datos_basicos["NVC Título 1"] = info["nvc_titulo1"]
            datos_basicos["NVC Título 2"] = info["nvc_titulo2"]
            datos_basicos["NVC Título 3"] = info["nvc_titulo3"]
            datos_basicos["Info Lectura Biblia"] = info["info_lectura_biblia"]
            datos_basicos["Info Estudio Libro"] = info["info_estudio_libro"]

            datos.append(datos_basicos)
            print(f"   ✅ Semana extraída: {info['semana']}")
            print(f"      • Lectura Biblia: {info['info_lectura_biblia']}") # Debug para verificar
        except Exception as e:
            print(f"   ❌ Error al procesar: {str(e)}")

        time.sleep(1)

    print("-" * 50)

    if datos:
        try:
            df = pd.DataFrame(datos)
            print(f"💾 Creando archivo Excel: {nombre_archivo}")
            with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name='Info-reunion', index=False)
            print(f"   ✅ Archivo creado exitosamente: {nombre_archivo}")
            return datos, nombre_archivo
        except Exception as e:
            print(f"   ❌ Error al crear el archivo Excel: {str(e)}")
            return None, None
    else:
        print("   ⚠️  No hay datos para guardar")
        return None, None

def descargar_archivo_colab(nombre_archivo):
    try:
        from google.colab import files
        print(f"📥 Iniciando descarga del archivo: {nombre_archivo}")
        files.download(nombre_archivo)
        print("✅ Descarga iniciada - revisa tus descargas")
    except ImportError:
        print(f"ℹ️  Entorno local: El archivo '{nombre_archivo}' está en la carpeta del script.")
    except Exception as e:
        print(f"❌ Error al descargar: {str(e)}")

# ========================================
# EJECUCIÓN AUTOMÁTICA
# ========================================
if __name__ == "__main__":
    print("="*60)
    print("🔄 EXTRACTOR VMC - EDWIN PROGRAMADOR")
    print("="*60)

    # 1. GENERAR URLS AUTOMÁTICAMENTE
    urls_generadas = generar_urls_por_mes(ANIO_BUSQUEDA, MES_BUSQUEDA)

    if urls_generadas:
        print(f"📋 Se encontraron {len(urls_generadas)} semanas para procesar.")

        # 2. PROCESAR
        datos_procesados, archivo_creado = procesar_urls_wol(urls_generadas)

        if datos_procesados and archivo_creado:
            print("\n" + "="*60)
            print("✅ PROCESO COMPLETADO")
            print("="*60)
            print(f"📊 Semanas: {len(datos_procesados)}")

            # 3. DESCARGAR
            descargar_archivo_colab(archivo_creado)
        else:
            print("\n❌ Hubo un error al generar el Excel.")
    else:
        print(f"\n❌ No se encontraron semanas (lunes) para {MES_BUSQUEDA}/{ANIO_BUSQUEDA}.")

    print("\n" + "="*60)

🔧 Instalando dependencias para Google Colab...
✅ Dependencias instaladas correctamente
🔄 EXTRACTOR VMC - EDWIN PROGRAMADOR
📅 Generando URLs para: 12/2025
📋 Se encontraron 5 semanas para procesar.
🔍 Extrayendo información de las páginas WOL...
--------------------------------------------------
📄 (1/5) Procesando: https://wol.jw.org/es/wol/dt/r4/lp-s/2025/12/01
   ✅ Semana extraída: 1-7 DE DICIEMBRE
      • Lectura Biblia: Is 5:1-12(thlección 5).
📄 (2/5) Procesando: https://wol.jw.org/es/wol/dt/r4/lp-s/2025/12/08
   ✅ Semana extraída: 8-14 DE DICIEMBRE
      • Lectura Biblia: Is 8:1-13(thlección 5).
📄 (3/5) Procesando: https://wol.jw.org/es/wol/dt/r4/lp-s/2025/12/15
   ✅ Semana extraída: 15-21 DE DICIEMBRE
      • Lectura Biblia: Is 10:1-14(thlección 11).
📄 (4/5) Procesando: https://wol.jw.org/es/wol/dt/r4/lp-s/2025/12/22
   ✅ Semana extraída: 22-28 DE DICIEMBRE
      • Lectura Biblia: Is 11:1-12(thlección 11).
📄 (5/5) Procesando: https://wol.jw.org/es/wol/dt/r4/lp-s/2025/12/29
   ✅ Sema

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Descarga iniciada - revisa tus descargas



# Copiar de Info-reunion a Formato

In [ ]:
# ========================================
# SCRIPT AUTOMÁTICO - UN SOLO MAPEO
# Copiar de Info-reunion a Formato
# ========================================

import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import range_boundaries
import re
import os

from google.colab import drive
drive.mount('/content/drive')

# Configuración
NOMBRE_ARCHIVO = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/Programación VMC_Septiembre-2025-2026.xlsx'

HOJA_ORIGEN = 'Info-reunion'
HOJA_DESTINO = 'Formato'
ESPACIADO_FILAS = 8

# MAPEO ÚNICO (con comentarios para completar)
MAPEO = {
    'Semana': 'B2',
    'Libro': 'I2',
    'Canción Inicial': 'D3',
    'Tesoros de la Biblia': 'C5',
    'Segunda Canción': 'D14',
    'Tercera Canción': 'D19',
    'Maestros Título 1': 'C9',
    'Maestros Título 2': 'C0',
    'Maestros Título 3': 'C11',
    'Maestros Título 4': 'C12',
    'NVC Título 1': 'C15',
    'NVC Título 2': 'C16',
    'NVC Título 3': 'C17',
    'Info Lectura Biblia': 'H7',
    'Info Estudio Libro': 'J17'
}

def copiar_filas_con_celdas_combinadas(archivo_excel, hoja_origen, hoja_destino,
                                       mapeo_celdas, espaciado=8):
    """Copia todas las filas de Info-reunion a Formato sin pedir nada al usuario."""
    try:
        print("\n🚀 Iniciando copia automática...")

        df_origen = pd.read_excel(archivo_excel, sheet_name=hoja_origen)
        print(f"   ✅ {len(df_origen)} filas encontradas en hoja origen")

        # Filas a copiar (todas)
        filas_a_copiar = df_origen

        wb = load_workbook(filename=archivo_excel)
        hoja_dest = wb[hoja_destino]

        def escribir_en_celda(hoja, celda_ref, valor):
            for rango_combinado in hoja.merged_cells.ranges:
                min_col, min_row, max_col, max_row = range_boundaries(str(rango_combinado))
                celda_col = hoja[celda_ref].column
                celda_row = hoja[celda_ref].row
                if min_col <= celda_col <= max_col and min_row <= celda_row <= max_row:
                    hoja.cell(row=min_row, column=min_col).value = valor
                    return
            hoja[celda_ref] = valor

        def extraer_coords(celda):
            match = re.match(r'([A-Z]+)(\d+)', celda.upper())
            if match:
                return match.group(1), int(match.group(2))
            return None, None

        columnas_validas = {col: celda for col, celda in mapeo_celdas.items()
                           if col in df_origen.columns and celda}

        contador_celdas = 0
        for idx, (_, fila) in enumerate(filas_a_copiar.iterrows()):
            desplazamiento = idx * (espaciado + 18)
            for columna, celda_base in columnas_validas.items():
                if pd.notna(fila[columna]):
                    letra, numero_base = extraer_coords(celda_base)
                    if letra and numero_base:
                        celda_destino = f"{letra}{numero_base + desplazamiento}"
                        escribir_en_celda(hoja_dest, celda_destino, fila[columna])
                        contador_celdas += 1

        wb.save(archivo_excel)
        wb.close()
        print(f"✅ {contador_celdas} celdas copiadas correctamente")
        print(f"💾 Archivo actualizado: {os.path.basename(archivo_excel)}")

    except Exception as e:
        print(f"❌ ERROR: {e}")
        import traceback
        traceback.print_exc()

# Punto de entrada automático
if __name__ == "__main__":
    copiar_filas_con_celdas_combinadas(NOMBRE_ARCHIVO, HOJA_ORIGEN, HOJA_DESTINO, MAPEO, ESPACIADO_FILAS)


Mounted at /content/drive

🚀 Iniciando copia automática...
   ✅ 4 filas encontradas en hoja origen
✅ 49 celdas copiadas correctamente
💾 Archivo actualizado: Programación VMC_Septiembre-2025-2026.xlsx


# Copiar información de la reunión a Formato

> Add blockquote



In [ ]:
# ========================================
# CÓDIGO MEJORADO Y AMIGABLE
# Copiar filas de Excel a celdas específicas
# ========================================

# Se importan todas las librerías necesarias al inicio
import pandas as pd
from openpyxl import load_workbook
import re
import os
import sys

## 1. CONFIGURACIÓN PRINCIPAL (AQUÍ AJUSTAS TODO)
# =================================================================

# Define la ruta base y el nombre del archivo.
# El script ajustará la ruta automáticamente si está en Colab.
RUTA_DRIVE = 'JW/Super VMC/Programación-VMC/Programador_VMC'
NOMBRE_ARCHIVO = 'Programación VMC_Septiembre-2025-2026.xlsx'

# Nombres de las hojas que vas a usar
HOJA_ORIGEN = 'BD ASIG'
HOJA_DESTINO = 'Formato'

# Espacio (en número de filas) entre cada bloque de datos pegado
ESPACIADO_FILAS = 8

# Mapeo de las columnas de origen a las celdas de destino.
# Revisa que los nombres de las columnas ('SEMANA', 'PRESIDENCIA', etc.)
# coincidan EXACTAMENTE con los de tu archivo Excel.
MAPEO_CELDAS = {
    'SEMANA': 'B2',
    'PRESIDENCIA': 'O2',
    'ORACIÓN': 'O3',
    'TESOROS DE LA BIBLIA': 'O5',
    'BUSQUEMOS PERLAS ESCONDIDAS': 'O6',
    'LECTURA DE LA BIBLIA': 'O7',
    'SMM ASIG 1 ESTUD': 'L9',
    'SMM ASIG 1 ACOMP': 'O9',
    'SMM ASIG 2 ESTUD': 'L10',
    'SMM ASIG 2 ACOMP': 'O10',
    'SMM ASIG 3 ESTUD': 'L11',
    'SMM ASIG 3 ACOMP': 'O11',
    'SMM ASIG 4 ESTUD': 'L12',
    'SMM ASIG 4 ACOMP': 'O12',
    'NVC PARTE 1': 'O15',
    'NVC PARTE 2': 'O16',
    'ESTUDIO LIBRO': 'O17',
    'LECTOR LIBRO': 'O18',
    'ORACIÓN FINAL': 'O19',
    'Info Lectura Biblia': 'H7',
    'Info Estudio Libro': 'J17'
}


## 2. LÓGICA DEL SCRIPT (No es necesario modificar esta parte)
# =================================================================

def copiar_filas_celdas_especificas(archivo_excel, hoja_origen, hoja_destino, mapeo_celdas, filtro=None, num_filas=None, espaciado=8):
    """Copia datos de una hoja a otra según la configuración."""
    try:
        print(f"\n{'='*60}\n🚀 INICIANDO PROCESO DE COPIA\n{'='*60}")
        if not os.path.exists(archivo_excel):
            raise FileNotFoundError(f"El archivo {os.path.basename(archivo_excel)} no se encontró en la ruta especificada.")

        print(f"1️⃣ Leyendo la hoja '{hoja_origen}'...")
        df_origen = pd.read_excel(archivo_excel, sheet_name=hoja_origen)
        print(f"   ✅ Se encontraron {len(df_origen)} filas.")

        df_filtrado = df_origen.copy()
        if filtro:
            columna, valor = filtro
            print(f"\n2️⃣ Aplicando filtro: {columna} = '{valor}'")
            if columna not in df_filtrado.columns:
                print(f"   ❌ ¡Error! La columna para filtrar '{columna}' no existe en la hoja de origen.")
                return
            df_filtrado = df_filtrado[df_filtrado[columna].astype(str).str.upper() == str(valor).upper()]
            if df_filtrado.empty:
                print(f"   ⚠️ No se encontraron filas que cumplan con el filtro.")
                return

        filas_a_copiar = df_filtrado
        if num_filas:
            print(f"\n3️⃣ Seleccionando las últimas {num_filas} filas...")
            filas_a_copiar = df_filtrado.tail(num_filas)

        print(f"\n✅ Se procesarán {len(filas_a_copiar)} filas.")
        if filas_a_copiar.empty:
            print("No hay filas para procesar. Proceso terminado.")
            return

        print("\n4️⃣ Abriendo archivo para escritura...")
        wb = load_workbook(filename=archivo_excel)
        if hoja_destino not in wb.sheetnames:
            print(f"   ❌ ¡Error! La hoja de destino '{hoja_destino}' no existe.")
            return
        hoja_dest = wb[hoja_destino]

        print("\n5️⃣ Copiando datos...")
        def extraer_coords(celda):
            match = re.match(r'([A-Z]+)(\d+)', celda.upper())
            return match.group(1), int(match.group(2))

        for idx, (_, fila) in enumerate(filas_a_copiar.iterrows()):
            desplazamiento = idx * (espaciado + 18)
            for columna, celda_base in mapeo_celdas.items():
                if columna in fila.index and pd.notna(fila[columna]):
                    letra, numero_base = extraer_coords(celda_base)
                    hoja_dest[f"{letra}{numero_base + desplazamiento}"] = fila[columna]

        print("\n6️⃣ Guardando cambios...")
        wb.save(archivo_excel)
        wb.close()
        print(f"\n🎉 ¡PROCESO COMPLETADO EXITOSAMENTE! 🎉")
        print(f"   Se han actualizado los datos en '{os.path.basename(archivo_excel)}'")

    except FileNotFoundError as e:
        print(f"\n❌ ERROR DE ARCHIVO: {e}")
    except Exception as e:
        print(f"\n❌ ERROR INESPERADO: {e}")
        import traceback
        traceback.print_exc()

## 3. BLOQUE DE EJECUCIÓN (INTERACTIVO)
# =================================================================

def ejecutar_proceso_interactivo():
    # --- Detección de entorno y configuración de ruta ---
    IN_COLAB = 'google.colab' in sys.modules
    archivo_excel = ""
    if IN_COLAB:
        print("📍 Entorno de Google Colab detectado.")
        try:
            from google.colab import drive
            drive.mount('/content/drive', force_remount=True)
            archivo_excel = os.path.join('/content/drive/MyDrive', RUTA_DRIVE, NOMBRE_ARCHIVO)
            print("✅ Google Drive conectado.")
        except Exception as e:
            print(f"❌ Error al conectar con Google Drive: {e}")
            return
    else:
        print("🖥️ Entorno local detectado.")
        archivo_excel = NOMBRE_ARCHIVO

    # --- Menú Interactivo ---
    while True:
        print(f"\n{'='*60}")
        print("MENU DE OPCIONES - ¿Qué deseas hacer?")
        print(f"{'='*60}")
        print("1. Filtrar por un mes específico (ej: AGOSTO, SEPTIEMBRE)")
        print("2. Copiar las últimas 'N' filas")
        print("3. Procesar TODAS las filas de la hoja de origen")
        print("4. Salir")

        opcion = input("\n👉 Elige una opción (1-4): ")

        if opcion == '1':
            mes = input("   Escribe el nombre del mes que quieres filtrar: ").upper()
            copiar_filas_celdas_especificas(
                archivo_excel, HOJA_ORIGEN, HOJA_DESTINO, MAPEO_CELDAS,
                filtro=('MES', mes), espaciado=ESPACIADO_FILAS
            )
            break
        elif opcion == '2':
            try:
                n_filas = int(input("   ¿Cuántas de las últimas filas quieres copiar?: "))
                copiar_filas_celdas_especificas(
                    archivo_excel, HOJA_ORIGEN, HOJA_DESTINO, MAPEO_CELDAS,
                    num_filas=n_filas, espaciado=ESPACIADO_FILAS
                )
            except ValueError:
                print("   ❌ Error: Por favor, introduce un número válido.")
            break
        elif opcion == '3':
            copiar_filas_celdas_especificas(
                archivo_excel, HOJA_ORIGEN, HOJA_DESTINO, MAPEO_CELDAS,
                espaciado=ESPACIADO_FILAS
            )
            break
        elif opcion == '4':
            print("\n👋 Proceso cancelado por el usuario. ¡Hasta luego!")
            break
        else:
            print("\n❌ Opción no válida. Por favor, elige un número del 1 al 4.")

# --- Ejecuta el proceso ---
ejecutar_proceso_interactivo()

📍 Entorno de Google Colab detectado.
Mounted at /content/drive
✅ Google Drive conectado.

MENU DE OPCIONES - ¿Qué deseas hacer?
1. Filtrar por un mes específico (ej: AGOSTO, SEPTIEMBRE)
2. Copiar las últimas 'N' filas
3. Procesar TODAS las filas de la hoja de origen
4. Salir

👉 Elige una opción (1-4): 1
   Escribe el nombre del mes que quieres filtrar: NOVIEMBRE

🚀 INICIANDO PROCESO DE COPIA
1️⃣ Leyendo la hoja 'BD ASIG'...
   ✅ Se encontraron 13 filas.

2️⃣ Aplicando filtro: MES = 'NOVIEMBRE'

✅ Se procesarán 4 filas.

4️⃣ Abriendo archivo para escritura...

5️⃣ Copiando datos...

6️⃣ Guardando cambios...

🎉 ¡PROCESO COMPLETADO EXITOSAMENTE! 🎉
   Se han actualizado los datos en 'Programación VMC_Septiembre-2025-2026.xlsx'


In [ ]:
!ls "/content/drive/MyDrive/Programación-VMC/"

ls: cannot access '/content/drive/MyDrive/Programación-VMC/': No such file or directory


# 📥 FASE 1: Extracción Automática (WOL)

In [2]:
# =============================================================================
#  🤖 ASISTENTE DE PROGRAMACIÓN VMC - FASE 1: EXTRACCIÓN DE DATOS (WOL)
# =============================================================================
#  Este programa se conecta a la Biblioteca en Línea (WOL), descarga la info
#  de las reuniones del mes seleccionado y la guarda en tu hoja 'Info-reunion'.
# =============================================================================

import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import re
import calendar
from datetime import datetime, timedelta
import time
import os
from google.colab import drive

# ==========================================
# 1. CONFIGURACIÓN INICIAL (¡SOLO CAMBIA ESTO!)
# ==========================================

# FECHA A PROCESAR
ANIO_BUSQUEDA = 2025
MES_BUSQUEDA = 12   # 12 = Diciembre

# RUTA DE TU ARCHIVO EXCEL EN DRIVE
RUTA_ARCHIVO = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/Programación VMC_Septiembre-2025-2026.xlsx'

# Headers para simular navegador (Evita bloqueos)
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# ==========================================
# 2. FUNCIONES DE EXTRACCIÓN (EL MOTOR)
# ==========================================

def generar_urls_por_mes(anio, mes):
    """Genera los enlaces de los lunes del mes seleccionado."""
    print(f"📅 Calculando semanas para: {mes}/{anio}...")
    urls = []
    cal = calendar.Calendar()
    for week in cal.monthdatescalendar(anio, mes):
        for day in week:
            # Si es Lunes (weekday 0) y pertenece al mes (o su semana toca el mes)
            if day.weekday() == 0 and (day.month == mes or (day + timedelta(days=6)).month == mes):
                url = f"https://wol.jw.org/es/wol/dt/r4/lp-s/{day.year}/{day.month:02d}/{day.day:02d}"
                if url not in urls:
                    urls.append(url)
    return urls

def extraer_informacion(url):
    """Visita WOL y saca los datos de una semana específica."""
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Estructura vacía
        info = {
            "semana": "", "libro": "", "cancion_inicial": "",
            "tesoros": "", "segunda_cancion": "", "tercera_cancion": "",
            "titulos_maestros": [], "nvc_titulo1": "", "nvc_titulo2": "", "nvc_titulo3": "",
            "info_lectura_biblia": "", "info_estudio_libro": ""
        }

        # --- A. Extracción Básica ---
        semana_elem = soup.find('h1', id='p1')
        if semana_elem: info["semana"] = semana_elem.get_text(strip=True)
        else:
            title = soup.find('title')
            if title: info["semana"] = title.get_text().split('|')[0].strip()

        libro_elem = soup.find('h2', id='p2')
        if libro_elem:
            strong_tags = libro_elem.find_all('strong')
            if strong_tags: info["libro"] = " ".join([s.get_text(strip=True) for s in strong_tags])
            else: info["libro"] = libro_elem.get_text(strip=True)

        tesoros_elem = soup.find(['h5', 'h4', 'h3'], id='p5')
        if tesoros_elem: info["tesoros"] = tesoros_elem.get_text(strip=True)

        # --- B. Canciones ---
        todas_h3 = soup.find_all('h3')
        canciones = []
        for h3 in todas_h3:
            txt = h3.get_text(strip=True)
            if "Canción" in txt:
                match = re.search(r'Canción\s+(\d+)', txt)
                if match:
                    c_str = f"Canción {match.group(1)}"
                    if c_str not in canciones: canciones.append(c_str)

        if len(canciones) >= 1: info["cancion_inicial"] = canciones[0]
        if len(canciones) >= 2: info["segunda_cancion"] = canciones[1]
        if len(canciones) >= 3: info["tercera_cancion"] = canciones[2]

        # --- C. Lectura de la Biblia (Lógica Robusta) ---
        parrafos_lectura = soup.find_all(lambda tag: tag.name == "p" and "(4 mins.)" in tag.text)
        for p in parrafos_lectura:
            texto_limpio = p.get_text(strip=True).replace('\xa0', ' ')
            match = re.search(r'\(\d+\s+mins?.*?\)\s*(.+)', texto_limpio)
            if match:
                posible_lectura = match.group(1).strip()
                if "th lección" in posible_lectura or re.search(r'\d+:', posible_lectura):
                    info["info_lectura_biblia"] = posible_lectura
                    break

        # --- D. Estudio Bíblico ---
        for h3 in soup.find_all('h3'):
            if "Estudio bíblico de la congregación" in h3.get_text(strip=True):
                siguiente = h3.find_next_sibling('div')
                if siguiente and siguiente.find('p'):
                    txt = siguiente.find('p').get_text(strip=True)
                    match = re.search(r'\(\d+\s+mins?.*?\)\s*(.+)', txt)
                    if match: info["info_estudio_libro"] = match.group(1).strip()
                    break

        if not info["info_estudio_libro"]:
             for div in soup.find_all('div'):
                p = div.find('p')
                if p and "(30 mins.)" in p.get_text():
                    txt = p.get_text(strip=True)
                    if any(x in txt.lower() for x in ['lfb', 'lmd', 'lvs', 'bt']):
                        match = re.search(r'\(\d+\s+mins?.*?\)\s*(.+)', txt)
                        if match: info["info_estudio_libro"] = match.group(1).strip()

        # --- E. Maestros y Vida ---
        maestros_node = soup.find('h2', string=re.compile("SEAMOS MEJORES MAESTROS"))
        vida_node = soup.find('h2', string=re.compile("NUESTRA VIDA CRISTIANA"))

        if maestros_node and vida_node:
            try:
                m_id = int(''.join(filter(str.isdigit, maestros_node.get('id', '0'))))
                v_id = int(''.join(filter(str.isdigit, vida_node.get('id', '9999'))))
                for h3 in soup.find_all('h3'):
                    h3_id_str = h3.get('id', '')
                    if not h3_id_str: continue
                    h3_val = int(''.join(filter(str.isdigit, h3_id_str)))
                    strong = h3.find('strong')
                    if not strong: continue
                    texto_parte = strong.get_text(strip=True)
                    if "Canción" in texto_parte: continue
                    if m_id < h3_val < v_id:
                        info["titulos_maestros"].append(texto_parte)
                    elif h3_val > v_id:
                        if "Estudio bíblico" not in texto_parte and "Palabras de conclusión" not in texto_parte:
                             if info["nvc_titulo1"] == "": info["nvc_titulo1"] = texto_parte
                             elif info["nvc_titulo2"] == "": info["nvc_titulo2"] = texto_parte
            except: pass

        if not info["nvc_titulo3"]:
             info["nvc_titulo3"] = "Estudio bíblico de la congregación"

        return info

    except Exception as e:
        print(f"❌ Error leyendo {url}: {e}")
        return None

# ==========================================
# 3. FUNCIÓN DE GUARDADO EN EXCEL (MEJORADA)
# ==========================================

def actualizar_info_reunion(ruta_archivo, datos_nuevos):
    """Guarda los datos buscando la primera fila vacía REAL (ignora formato basura)."""
    print(f"\n💾 Guardando datos en: {os.path.basename(ruta_archivo)}...")

    try:
        book = load_workbook(ruta_archivo)
        df_nuevos = pd.DataFrame(datos_nuevos)

        # Preparar DataFrame de Exportación
        df_export = pd.DataFrame()
        df_export['Semana'] = df_nuevos['semana']
        df_export['Libro'] = df_nuevos['libro']
        df_export['Canción Inicial'] = df_nuevos['cancion_inicial']
        df_export['Tesoros de la Biblia'] = df_nuevos['tesoros']
        df_export['Segunda Canción'] = df_nuevos['segunda_cancion']
        df_export['Tercera Canción'] = df_nuevos['tercera_cancion']

        # Maestros (hasta 4 partes)
        maestros_list = df_nuevos['titulos_maestros'].tolist()
        for i in range(1, 5):
            col_name = f'Maestros Título {i}'
            valores = []
            for lista in maestros_list:
                valores.append(lista[i-1] if i <= len(lista) else "")
            df_export[col_name] = valores

        df_export['NVC Título 1'] = df_nuevos['nvc_titulo1']
        df_export['NVC Título 2'] = df_nuevos['nvc_titulo2']
        df_export['NVC Título 3'] = df_nuevos['nvc_titulo3']
        df_export['Info Lectura Biblia'] = df_nuevos['info_lectura_biblia']
        df_export['Info Estudio Libro'] = df_nuevos['info_estudio_libro']

        # Verificar hoja
        if 'Info-reunion' not in book.sheetnames:
            print("   ⚠️ La hoja no existía. Creándola...")
            with pd.ExcelWriter(ruta_archivo, engine='openpyxl', mode='a') as writer:
                df_export.to_excel(writer, sheet_name='Info-reunion', index=False)
        else:
            ws = book['Info-reunion']

            # --- CORRECCIÓN CLAVE: BUSCAR LA PRIMERA FILA VACÍA REAL ---
            # Empezamos en la fila 2 (asumiendo que la 1 tiene los títulos)
            fila_inicio = 2
            while ws[f'A{fila_inicio}'].value is not None:
                fila_inicio += 1

            print(f"   ℹ️  Primera fila vacía encontrada: {fila_inicio}")

            # Filtrar duplicados (leemos lo que ya existe)
            # Nota: Leemos solo la columna A para ser rápidos
            semanas_existentes = []
            for row in ws.iter_rows(min_row=2, max_row=fila_inicio-1, min_col=1, max_col=1, values_only=True):
                if row[0]: semanas_existentes.append(str(row[0]).strip())

            # Filtramos el DataFrame nuevo
            nuevas_filas = df_export[~df_export['Semana'].isin(semanas_existentes)]

            if nuevas_filas.empty:
                print("   ℹ️  Todas las semanas ya estaban guardadas. No se hicieron cambios.")
            else:
                # Escribimos fila por fila en la posición correcta
                filas_datos = dataframe_to_rows(nuevas_filas, index=False, header=False)

                for i, row_data in enumerate(filas_datos):
                    current_row = fila_inicio + i
                    for j, value in enumerate(row_data):
                        # j+1 porque Excel empieza columnas en 1
                        ws.cell(row=current_row, column=j+1, value=value)

                book.save(ruta_archivo)
                print(f"   ✅ ¡Éxito! Se agregaron {len(nuevas_filas)} semanas nuevas empezando en la fila {fila_inicio}.")

    except Exception as e:
        print(f"❌ Error al guardar en Excel: {e}")

# ==========================================
# 4. EJECUCIÓN PRINCIPAL
# ==========================================

if __name__ == "__main__":
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

    print("\n" + "="*60)
    print(f"   🚀 INICIANDO EXTRACCIÓN DE DATOS WOL ({MES_BUSQUEDA}/{ANIO_BUSQUEDA})")
    print("="*60)

    urls = generar_urls_por_mes(ANIO_BUSQUEDA, MES_BUSQUEDA)
    print(f"📋 Se encontraron {len(urls)} semanas teocráticas.")

    datos_recolectados = []
    print("-" * 60)
    for i, url in enumerate(urls, 1):
        print(f"Processing ({i}/{len(urls)}): {url} ...", end=" ")
        info = extraer_informacion(url)
        if info:
            print("✅ Hecho.")
            print(f"   [Semana: {info['semana']} | Lectura: {info['info_lectura_biblia']}]")
            datos_recolectados.append(info)
        else:
            print("❌ Falló.")
        time.sleep(1)

    if datos_recolectados:
        print("-" * 60)
        actualizar_info_reunion(RUTA_ARCHIVO, datos_recolectados)
        print("\n🎉 FASE 1 COMPLETADA CORRECTAMENTE")
        print("👉 PRÓXIMO PASO: Abre tu Excel, ve a la hoja 'BD ASIG' y asigna los nombres.")
    else:
        print("\n❌ No se extrajeron datos.")


   🚀 INICIANDO EXTRACCIÓN DE DATOS WOL (12/2025)
📅 Calculando semanas para: 12/2025...
📋 Se encontraron 5 semanas teocráticas.
------------------------------------------------------------
Processing (1/5): https://wol.jw.org/es/wol/dt/r4/lp-s/2025/12/01 ... ✅ Hecho.
   [Semana: 1-7 DE DICIEMBRE | Lectura: Is 5:1-12(thlección 5).]
Processing (2/5): https://wol.jw.org/es/wol/dt/r4/lp-s/2025/12/08 ... ✅ Hecho.
   [Semana: 8-14 DE DICIEMBRE | Lectura: Is 8:1-13(thlección 5).]
Processing (3/5): https://wol.jw.org/es/wol/dt/r4/lp-s/2025/12/15 ... ✅ Hecho.
   [Semana: 15-21 DE DICIEMBRE | Lectura: Is 10:1-14(thlección 11).]
Processing (4/5): https://wol.jw.org/es/wol/dt/r4/lp-s/2025/12/22 ... ✅ Hecho.
   [Semana: 22-28 DE DICIEMBRE | Lectura: Is 11:1-12(thlección 11).]
Processing (5/5): https://wol.jw.org/es/wol/dt/r4/lp-s/2025/12/29 ... ✅ Hecho.
   [Semana: 29 DE DICIEMBRE DE 2025 A 4 DE ENERO DE 2026 | Lectura: Is 16:1-14(thlección 10).]
----------------------------------------------------

# 🚀 FASE 2: Fusión y Generación del Formato

In [ ]:
# =============================================================================
#  🚀 ASISTENTE DE PROGRAMACIÓN VMC - FASE 2: FUSIÓN Y GENERACIÓN (FORMATO)
# =============================================================================
#  Este script actúa como el "Constructor" del programa final. Su trabajo es:
#
#  1. LEER la información espiritual que descargaste en la Fase 1 (Hoja 'Info-reunion').
#  2. LEER los nombres de los hermanos que asignaste manualmente (Hoja 'BD ASIG').
#  3. BUSCAR las semanas que coincidan en ambas listas.
#  4. FUSIONAR toda la información y PEGARLA ordenadamente en la hoja 'Formato'.
#
#  NOTA: Este proceso respeta las celdas combinadas y el diseño de tu formato.
# =============================================================================

import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import range_boundaries
import re
import os
from google.colab import drive

# ==========================================
# 1. CONFIGURACIÓN (¡AJUSTA ESTO SI CAMBIA!)
# ==========================================

# RUTA EXACTA DE TU ARCHIVO EXCEL EN DRIVE
ARCHIVO_VMC = '/content/drive/MyDrive/JW/Super VMC/Programación-VMC/Programador_VMC/Programación VMC_Septiembre-2025-2026.xlsx'

# CONFIGURACIÓN DE ESPACIADO VISUAL
# ¿Cuántas filas ocupa una semana completa en la hoja 'Formato' incluyendo el espacio vacío hasta la siguiente?
# (Basado en tu formato: 18 filas de reunión + 8 filas de espacio = 26)
FILAS_POR_BLOQUE = 26

# ==========================================
# 2. MAPEO MAESTRO (DÓNDE VA CADA DATO)
# ==========================================
# Estructura: 'NOMBRE_COLUMNA_EN_DATOS': 'CELDA_DESTINO_EN_FORMATO'

MAPEO_TOTAL = {
    # --- A. DATOS DE WOL (Vienen de Fase 1 / Info-reunion) ---
    'Semana': 'B2',
    'Libro': 'I2',
    'Canción Inicial': 'D3',
    'Tesoros de la Biblia': 'C5',       # Título del discurso
    'Info Lectura Biblia': 'H7',        # Texto bíblico (ej: Is 11:1-12)
    'Segunda Canción': 'D14',
    'Tercera Canción': 'D19',
    'Info Estudio Libro': 'J17',        # Material estudio (ej: lfb lección 32)

    # Seamos Mejores Maestros (Títulos de las partes)
    'Maestros Título 1': 'C9',
    'Maestros Título 2': 'C10',
    'Maestros Título 3': 'C11',
    'Maestros Título 4': 'C12',

    # Vida Cristiana (Títulos de las partes)
    'NVC Título 1': 'C15',
    'NVC Título 2': 'C16',
    # 'NVC Título 3': 'C17',            # (Opcional: suele ser el Estudio Bíblico)

    # --- B. DATOS HUMANOS (Vienen de Fase 2 / BD ASIG) ---
    'PRESIDENCIA': 'O2',
    'ORACIÓN': 'O3',
    'LECTURA DE LA BIBLIA': 'O7',       # Nombre del estudiante

    # Asignaciones: Seamos Mejores Maestros (Nombres)
    'SMM ASIG 1 ESTUD': 'L9',
    'SMM ASIG 1 ACOMP': 'O9',
    'SMM ASIG 2 ESTUD': 'L10',
    'SMM ASIG 2 ACOMP': 'O10',
    'SMM ASIG 3 ESTUD': 'L11',
    'SMM ASIG 3 ACOMP': 'O11',
    'SMM ASIG 4 ESTUD': 'L12',
    'SMM ASIG 4 ACOMP': 'O12',

    # Asignaciones: Vida Cristiana (Nombres)
    'NVC PARTE 1': 'O15',               # Nombre hermano parte 1
    'NVC PARTE 2': 'O16',               # Nombre hermano parte 2
    'ESTUDIO LIBRO': 'O17',             # Conductor
    'LECTOR LIBRO': 'O18',              # Lector
    'ORACIÓN FINAL': 'O19'
}

# ==========================================
# 3. FUNCIONES AUXILIARES (HERRAMIENTAS)
# ==========================================

def escribir_en_celda_segura(hoja, celda_ref, valor):
    """
    Escribe en una celda asegurándose de no romper celdas combinadas.
    Si la celda 'C5' está combinada con 'D5', escribe en la celda principal.
    """
    if pd.isna(valor): return # Si el dato está vacío, no hacemos nada

    # Verificar si la celda destino es parte de un rango combinado
    for rango_combinado in hoja.merged_cells.ranges:
        min_col, min_row, max_col, max_row = range_boundaries(str(rango_combinado))
        celda_obj = hoja[celda_ref]

        # Si cae dentro de un rango combinado
        if (min_col <= celda_obj.column <= max_col) and (min_row <= celda_obj.row <= max_row):
            # Escribir en la celda superior-izquierda (la 'dueña' del rango)
            hoja.cell(row=min_row, column=min_col).value = valor
            return

    # Si no está combinada, escribir normalmente
    hoja[celda_ref] = valor

def extraer_coords(celda):
    """Convierte 'B2' en ('B', 2) para poder sumar filas."""
    match = re.match(r'([A-Z]+)(\d+)', celda.upper())
    if match:
        return match.group(1), int(match.group(2))
    return None, None

# ==========================================
# 4. PROCESO DE FUSIÓN (EL CONSTRUCTOR)
# ==========================================

def ejecutar_fusion_vmc():
    print("\n" + "="*60)
    print(" 🏗️  INICIANDO FUSIÓN: Info-Reunion + BD ASIG -> Formato")
    print("="*60)

    try:
        # 1. Cargar las hojas de Excel
        print("1️⃣  Leyendo tus hojas de Excel...")
        if not os.path.exists(ARCHIVO_VMC):
            print(f"❌ ERROR: No encuentro el archivo en: {ARCHIVO_VMC}")
            return

        df_info = pd.read_excel(ARCHIVO_VMC, sheet_name='Info-reunion')
        df_asig = pd.read_excel(ARCHIVO_VMC, sheet_name='BD ASIG')

        print(f"    • Info-reunion: {len(df_info)} filas encontradas.")
        print(f"    • BD ASIG:      {len(df_asig)} filas encontradas.")

        # 2. Normalizar la columna clave (La 'Llave Maestra')
        # Quitamos espacios y ponemos mayúsculas para que "3-9 Noviembre" coincida con "3-9 NOVIEMBRE"
        df_info['Clave'] = df_info['Semana'].astype(str).str.strip().str.upper()
        df_asig['Clave'] = df_asig['SEMANA'].astype(str).str.strip().str.upper()

        # 3. FUSIÓN (MERGE)
        # Unimos las dos tablas usando la 'Clave' (Semana). Solo guarda las que coincidan.
        print("\n2️⃣  Fusionando información...")
        df_completo = pd.merge(df_info, df_asig, on='Clave', how='inner')

        total_semanas = len(df_completo)

        if total_semanas == 0:
            print("   ⚠️  ALERTA: No encontré ninguna semana coincidente.")
            print("      Sug: Revisa que la columna 'Semana' en ambas hojas esté escrita exactamente igual.")
            return
        else:
            print(f"   ✅ ¡Éxito! Tengo {total_semanas} semanas listas para imprimir en el formato.")

        # 4. Escribir en la hoja 'Formato'
        print("\n3️⃣  Escribiendo en la hoja 'Formato'...")
        wb = load_workbook(ARCHIVO_VMC)
        ws = wb['Formato']

        for idx, row in df_completo.iterrows():
            # Calcular cuánto debemos bajar para escribir este bloque
            desplazamiento = idx * FILAS_POR_BLOQUE
            semana_actual = row['Semana'] # Nombre original para mostrar

            print(f"    • Procesando semana: {semana_actual}")

            # Recorrer el MAPEO y pegar dato por dato
            for columna_origen, celda_base in MAPEO_TOTAL.items():

                # Verificamos si esa columna existe en nuestros datos fusionados
                if columna_origen in row:
                    dato = row[columna_origen]

                    # Calcular la nueva celda destino (Ej: B2 -> B28 -> B54...)
                    letra, fila_base = extraer_coords(celda_base)
                    if letra:
                        nueva_celda = f"{letra}{fila_base + desplazamiento}"
                        escribir_en_celda_segura(ws, nueva_celda, dato)

        # 5. Guardar Cambios
        wb.save(ARCHIVO_VMC)
        print("\n" + "="*60)
        print(f"🎉 ¡PROCESO TERMINADO! Tu programación está lista.")
        print(f"📁 Archivo actualizado: {os.path.basename(ARCHIVO_VMC)}")
        print("="*60)

    except Exception as e:
        print(f"\n❌ OCURRIÓ UN ERROR INESPERADO: {e}")
        import traceback
        traceback.print_exc()

# ==========================================
# 5. EJECUCIÓN
# ==========================================
if __name__ == "__main__":
    # Montar Drive si es necesario
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

    ejecutar_fusion_vmc()